# Getting random sample

Rfam database contains 3115 RNA-sequences

In [3]:
import os
import random
from random import randint

## Choose number of families

In [4]:
# amount_of_fam is how much of families you want to download from Rfam

gen_range, numbers, amount_of_fam = 3115, [], 10 #int(input('Print number RANGE')), [], int(input('Print size of the sample'))

while amount_of_fam > 0:
    number = random.randint(1, gen_range)
    if number not in numbers:
        numbers.append(number)
        amount_of_fam -= 1

## Downloading files

In [5]:
with open("files_to_download.txt", 'w') as out:
    for num in numbers:
        out.write('ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF' + ((5 - len(str(num))) * '0' + str(num)) + '.fa.gz' + '\n')
! wget -i "files_to_download.txt" -P "../data/families_seq/"

--2019-05-20 15:10:53--  ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF00204.fa.gz
           => ‘../data/families_seq/RF00204.fa.gz’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.192.4
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.192.4|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/Rfam/14.1/fasta_files ... done.
==> SIZE RF00204.fa.gz ... 13965
==> PASV ... done.    ==> RETR RF00204.fa.gz ... done.
Length: 13965 (14K) (unauthoritative)

RF00204.fa.gz       100%[===================>]  13.64K  --.-KB/s    in 0.07s   

2019-05-20 15:10:54 (209 KB/s) - ‘../data/families_seq/RF00204.fa.gz’ saved [13965]

--2019-05-20 15:10:54--  ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF01100.fa.gz
           => ‘../data/families_seq/RF01100.fa.gz’
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.192.4|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST .

##### Unpacking and deleting archives

In [6]:
os.chdir("../data/families_seq/")
! gunzip *.gz

##### Here we determine the number of seq form each family (<=5); you can change numbers if you need

In [7]:
! grep -c "^>" *.fa >> number_of_seqs.txt
with open("number_of_seqs.txt", 'r') as fname:
    num_of_seq = []
    for num in fname:
        num_of_seq.append(int(num.strip().split(":")[1]))
    need_num = min(num_of_seq) if min(num_of_seq) <= 10 else 10 # 5 else 5

In [ ]:
'''
YOU CAN USE THIS CELL IF YOU WANT TO MAKE TWO SAMPLES 
FROM ONE SET OF FAMILIES
'''
check_list = []
with open('../data/sample_first_70.fasta') as fname:
    for string in fname:
        if ">" in string:
            check_list.append(string.strip()[1:-2])

##### Now we make our sample :) WITH UNIQUE-CHECKER

In [8]:
'''
Here you can get random sample with unique sequences.
It is important to note that if the family randomly selected in the previous 
step contains too many duplicate sequences, then its representation in the 
sample will be less than we expected. If the family is represented too poorly,
delete the downloaded files (see below) and repeat the random getting of families
'''
need_num = 10
# >RNU6-511P_ENSG00000200366.1,0
import glob
from random import sample
from Bio import SeqIO
from uuid import uuid1

out = open("sample.fasta", 'a')
fam_name = 0

for file in glob.glob("./*.fa"):
    fam = file[2:-3]
    #record = SeqIO.to_dict(SeqIO.parse(file, "fasta"))
    record, seq_check = {}, []
    for rec in SeqIO.parse(file, "fasta"):
        rec.id = rec.id.split("/")[0]
        #print(rec.id, rec.seq)
        if rec.id not in record and str(rec.seq) not in seq_check and len(rec.seq) <= 900 and len(rec.seq) >= 70:
            # print(rec.id.split("/")[0])
            record[rec.id] = str(rec.seq)
            seq_check.append(str(rec.seq))
        #print(record)
    
    if need_num > len(record):
        end = len(record)
    else:
        end = need_num
        
    sp = random.sample(record.keys(), end)
    
    for seq_name in sp:
        out.write(">" + fam + "_" + str(seq_name).split("/")[0] + "," + str(fam_name) + '\n')
        out.write(str(record[seq_name]) + '\n')
    fam_name += 1
    
out.close()

In [9]:
! grep -c ",0" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",1" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",2" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",3" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",4" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",5" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",6" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",7" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",8" sample.fasta >> HOWMUCHFAM.txt
! grep -c ",9" sample.fasta >> HOWMUCHFAM.txt

In [10]:
! grep -c ">" sample.fasta

393


##### Making genelabel file

In [11]:
from Bio import SeqIO
with open("fam70_test.txt", 'a') as out:
    for record in SeqIO.parse("sample.fasta", "fasta"):
        iden = str(record.id).split(",")
        out.write(iden[0] + ":" + iden[1] + '\n')

In [12]:
! grep -c ":" genelabel_sample.txt # You can know this file as *fam70_test.txt*

grep: genelabel_sample.txt: No such file or directory
